# CDN Analysis Test for FEB 2021 DATA

# Introduction:

1.	What is the Cost per Passenger(CPP)?
2.	What is the total pax?
<br>    Answered
3.	What is the number of pax for each class?
<br>    Answered
4.	What is the total expenses?
<br>    Answered
5.	What is the total expenses per class?
<br>   Answered
6.	What is the total load factor?
<br>    Answered
7.	 What is the load factor per class? 
<br>    Answered
8.	Which station has the highest number of pax?
9.	Which station has the highest total price compared to unit price?
10.	Which station has the lowest number of pax compared to capacity?
11.	What is the number of pax in relation to time-of-flight date and hour?
12.	What is the impact of monthly changes?
13.	What are the top 10 highest volume items? 
14.	What are the top 10 highest expense items?
15.	What are the top 5 highest passenger count sectors?
16.	What is the highest order volume based on time?
17.	What is the correlation/impact of the load factor on cpp?
18.	What is the handling charge per route?
<br>    Answered
19.	How does the seasonally impact on cpp?
20.	What is the DOM cpp by menu cycle?
21.	SACC station ordered by volume and expenses?
22. what is the total handling charges?
<br>    Answered
23. what is the handling charges per Air Carft?
24. What are the monthly changes and their impact?

## Data Cleaning:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
%matplotlib inline

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# Initiate Data Frame for the current month and previous month
df = pd.read_excel("AVIF Feb 2021.xlsb", engine='pyxlsb')
df_p = pd.read_excel("AVIF Jan 2021.xlsb", engine='pyxlsb')

In [ ]:
del_col = ['MAIN_SBD_CODE', 'AIRCRAFT_TYPE', 'TOP5', 'ROUTE_TYPE', 'CUST_NO', 'ROUTE', 'LEGNO', 'SR_FLT_NO', 'SR_SUFFIX', 'SR_FROM_CITY', 'SR_TO_CITY', 'SERVICE_NO', 'FPM_CODE', 'MENU_CODE', 'MENU_DESC', 'QTY', 'VAT_PRICE', 'MENU_CYCLE']
df.drop(del_col,axis=1, inplace=True)
df_p.drop(del_col,axis=1, inplace=True)

In [ ]:
# print first 5 rows of data
df.head()

In [ ]:
df_p.head()

In [ ]:
# drop international flights colunn
df_p.drop(df_p[df_p['TYPE_CODE'] == 'C1'].index, inplace=True)
df.drop(df[df['TYPE_CODE'] == 'C1'].index, inplace=True)

In [ ]:
# drop the not related class values to the passenger count
df.drop(df[df.CLASS_CODE.isin(['CA', 'CO', 'CP', 'PX', 'PXC','SM'])].index, inplace=True)
df_p.drop(df_p[df_p.CLASS_CODE.isin(['CA', 'CO', 'CP', 'PX', 'PXC','SM'])].index, inplace=True)

In [ ]:
# drop rows with item descreption equals to LAVATORY DISINFECTANT 750ML (SV) 
df_p.drop(df_p[df_p['ITEM_DESCRIPTION'] == 'LAVATORY DISINFECTANT 750ML (SV)'].index, inplace=True)
df.drop(df[df['ITEM_DESCRIPTION'] == 'LAVATORY DISINFECTANT 750ML (SV)'].index, inplace=True)

In [ ]:
# Convert the date to datetime64
df_p['DBL_DATE'] = pd.to_datetime(df_p['DBL_DATE'] - 25569, unit = 'D')

In [ ]:
df['DBL_DATE']  = pd.to_datetime(df['DBL_DATE'] - 25569, unit = 'D')

In [ ]:
df.info()

In [ ]:
df_p.info()

In [ ]:
# Convert specs_no to string 
df_p['SPECS_NO'] = df_p['SPECS_NO'].astype(str)

In [ ]:
df['SPECS_NO'].fillna(0, inplace=True)
df['SPECS_NO'] = df['SPECS_NO'].astype('int32')

In [ ]:
df['SPECS_NO'] = df['SPECS_NO'].astype(str)

In [ ]:
# Convert price to integer 
df_p['PRICE'] = df_p['PRICE'].astype(int)
df['PRICE'] = df['PRICE'].astype(int)

In [ ]:
# change numerical columns to int32 and float32 to reduce memory usage 
df['NO_PAX'] = df['NO_PAX'].astype('int32')
df_p['NO_PAX'] = df_p['NO_PAX'].astype('int32')

df['CAPACITY'] = df['CAPACITY'].astype('int32')
df_p['CAPACITY'] = df_p['CAPACITY'].astype('int32')

df['UNIT_PRICE'] = df['UNIT_PRICE'].astype('float32')
df_p['UNIT_PRICE'] = df_p['UNIT_PRICE'].astype('float32')

In [ ]:
# Change CDN_NO to String
df['CDN_NO'] = df['CDN_NO'].astype(str)
df_p['CDN_NO'] = df_p['CDN_NO'].astype(str)

In [ ]:
#Check memory usage for df
df.info(memory_usage="deep")

In [ ]:
#Check memory usage for df_p
df_p.info(memory_usage="deep")

### Import and Clean Summer Schedule: 

This step will help in categorizing flights as per meal policy

In [ ]:
# import the summer schedule to get block time
summerSchedule = pd.read_excel('summer_schedule.xlsx')

In [ ]:
# Print first five rows
summerSchedule.head()

In [ ]:
# rename sector columns to be aligned with the other dataframe to be joined with 
summerSchedule.rename(columns={"Dept Arp": "FROM_CITY", "Arvl Arp": "TO_CITY"}, inplace=True)

In [ ]:
# drop unused columns
summerSchedule = summerSchedule.drop(columns=['Eff Date', 'Dis Date','Freq','Dept Time','Arrv Time','Subfleet', 'Flt Desg'])

In [ ]:
# Print first five rows
summerSchedule.head()

In [ ]:
# remove duplicates to ensure merge will become 1:m
summerSchedule.drop_duplicates(subset=['FROM_CITY', 'TO_CITY'], inplace=True)

In [ ]:
# print new df informations
summerSchedule.info()

In [ ]:
#perform the join of the dataframes
df = df.merge(summerSchedule, on=['FROM_CITY','TO_CITY'],how = 'inner')
df_p = df_p.merge(summerSchedule, on=['FROM_CITY','TO_CITY'],how = 'inner')

In [ ]:
# Convert block time into minutes
df_p['Block Time'] =  pd.to_datetime(df_p['Block Time'].astype(str))
df['Block Time'] =  pd.to_datetime(df['Block Time'].astype(str))

In [ ]:
df_p['Block Time'] = df_p['Block Time'].dt.hour*60+df_p['Block Time'].dt.minute
df['Block Time'] = df['Block Time'].dt.hour*60+df['Block Time'].dt.minute

## Create a paxFarme:

In [ ]:
#Create a copy of the main data farme and store it in the variable df_copy
df_copy = df.copy()
df_p_copy = df_p.copy()

In [ ]:
# make an instance of df_copy and call it paxFrame
paxFrame = df_copy[['CDN_NO', 'DBL_DATE', 'FLT_NO', 'TYPE_CODE', 'FROM_CITY', 'TO_CITY', 'CAPACITY', 'NO_PAX', 'CLASS_CODE', 'Block Time']]
paxFrameP = df_p_copy[['CDN_NO', 'DBL_DATE', 'FLT_NO', 'TYPE_CODE', 'FROM_CITY', 'TO_CITY', 'CAPACITY', 'NO_PAX', 'CLASS_CODE', 'Block Time']]

In [ ]:
#check the information of paxFrame
paxFrame.info()

In [ ]:
#check the information of paxFrameP
paxFrameP.info()

In [ ]:
# Check the number of passengers
paxFrame['NO_PAX'].sum()

In [ ]:
# Check the number of passengers
paxFrameP['NO_PAX'].sum()

In [ ]:
# the number of passenger shown above is un realistic becuase of duplicated 'NO_PAX' which is related to menu items
# Therefore this code is used to remove duplicated rows
paxFrame.drop_duplicates(subset=['FROM_CITY', 'TO_CITY','DBL_DATE','CLASS_CODE','FLT_NO', 'NO_PAX'], inplace=True)

In [ ]:
# the number of passenger shown above is un realistic becuase of duplicated 'NO_PAX' which is related to menu items
# Therefore this code is used to remove duplicated rows
paxFrameP.drop_duplicates(subset=['FROM_CITY', 'TO_CITY', 'DBL_DATE','CLASS_CODE','FLT_NO', 'NO_PAX'], inplace=True)

In [ ]:
# Check the paxFrame after removing duplicates
paxFrame.info()

In [ ]:
# Check the paxFrame after removing duplicates
paxFrameP.info()

## Number of Passengers:

#### Total Number of Passengers:

In [ ]:
paxCountP = paxFrameP['NO_PAX'].sum()
print('Total number of passenger for the previous month {:,}'.format(paxCountP))

In [ ]:
paxCount = paxFrame['NO_PAX'].sum()
print('Total number of passenger for the current month {:,}'.format(paxCount))

##### Number of Passengers per Class for Previous Month:

In [ ]:
fc_count_p = paxFrameP['NO_PAX'][paxFrameP['CLASS_CODE'] == 'FC'].sum()
fc_weight_p = fc_count_p/paxCountP
print('First class passengers count equals {:,} PAX, and its weight is {}'.format(fc_count_p, np.round(fc_weight_p, 3)))

bc_count_p = paxFrameP['NO_PAX'][paxFrameP['CLASS_CODE'] == 'JC'].sum()
bc_weight_p = bc_count_p/paxCountP
print('Business class passengers count equals {:,} PAX, and its weight is {}'.format(bc_count_p, np.round(bc_weight_p, 2)))

yc_count_p = paxFrameP['NO_PAX'][paxFrameP['CLASS_CODE'] == 'YC'].sum()
yc_weight_p = yc_count_p/paxCountP
print('Economy class passengers count equals {:,} PAX, and its weight is {}'.format(yc_count_p, np.round(yc_weight_p, 2)))

##### Number of Passengers per Class for Current Month:

In [ ]:
fc_count = paxFrame['NO_PAX'][paxFrame['CLASS_CODE'] == 'FC'].sum()
fc_weight = fc_count/paxCount
print('First class passengers count equals {:,} PAX, and its weight is {}'.format(fc_count, np.round(fc_weight, 3)))

bc_count = paxFrame['NO_PAX'][paxFrame['CLASS_CODE'] == 'JC'].sum()
bc_weight = bc_count/paxCount
print('Business class passengers count equals {:,} PAX, and its weight is {}'.format(bc_count, np.round(bc_weight, 2)))

yc_count = paxFrame['NO_PAX'][paxFrame['CLASS_CODE'] == 'YC'].sum()
yc_weight = yc_count/paxCount
print('Economy class passengers count equals {:,} PAX, and its weight is {}'.format(yc_count, np.round(yc_weight, 2)))

### Seat Capacity:

#### Total Capacity:

In [ ]:
seatCapacityP = paxFrameP['CAPACITY'].sum()
print('Total capacity for domestic flights for previous month equals {:,} seat.'.format(seatCapacityP))

In [ ]:
seatCapacity = paxFrame['CAPACITY'].sum()
print('Total capacity for domestic flights for current month equals {:,} seat.'.format(seatCapacity))

##### Capacity per Class Previous Month:

In [ ]:
fc_capacity_p = paxFrameP['CAPACITY'][paxFrameP['CLASS_CODE'] == 'FC'].sum()
print('First class seat capacity equals {:,} seat.'.format(fc_capacity_p))

bc_capacity_p  = paxFrameP['CAPACITY'][paxFrameP['CLASS_CODE'] == 'JC'].sum()
print('Business class seat capacity equals {:,} seat.'.format(bc_capacity_p))

yc_capacity_p  = paxFrameP['CAPACITY'][paxFrameP['CLASS_CODE'] == 'YC'].sum()
print('Economy class seat capacity equals {:,} seat.'.format(yc_capacity_p))

##### Capacity per Class Current month:

In [ ]:
fc_capacity = paxFrame['CAPACITY'][paxFrame['CLASS_CODE'] == 'FC'].sum()
print('First class seat capacity equals {:,} seat.'.format(fc_capacity))

bc_capacity  = paxFrame['CAPACITY'][paxFrame['CLASS_CODE'] == 'JC'].sum()
print('Business class seat capacity equals {:,} seat.'.format(bc_capacity))

yc_capacity  = paxFrame['CAPACITY'][paxFrame['CLASS_CODE'] == 'YC'].sum()
print('Economy class seat capacity equals {:,} seat.'.format(yc_capacity))

### Load Factor:

#### Total Load Factor:

In [ ]:
loadFactor = (paxCountP/seatCapacityP)*100
print('domestic flights load factor for previous month {}%'.format(np.round(loadFactor,2)))

In [ ]:
loadFactor = (paxCount/seatCapacity)*100
print('domestic flights load factor for current month {}%'.format(np.round(loadFactor,2)))

##### Load Factor per Class Previous Month:

In [ ]:
fc_lf_p = (fc_count_p/fc_capacity_p)*100
print('First class load factor equals {}%'.format(np.round(fc_lf_p, 2)))

bc_lf_p = (bc_count_p/bc_capacity_p)*100
print('Business class load factor equals {}%'.format(np.round(bc_lf_p, 2)))

yc_lf_p  = (yc_count_p/yc_capacity_p)*100
print('Economy class load factor equals {}%'.format(np.round(yc_lf_p, 2)))

##### Load Factor per Class Current Month:

In [ ]:
fc_lf = (fc_count/fc_capacity)*100
print('First class load factor equals {}%'.format(np.round(fc_lf, 2)))

bc_lf = (bc_count/bc_capacity)*100
print('Business class load factor equals {}%'.format(np.round(bc_lf, 2)))

yc_lf  = (yc_count/yc_capacity)*100
print('Economy class load factor equals {}%'.format(np.round(yc_lf, 2)))

# Expenses:

### Total Expenses:

In [ ]:
domExpP = df_p['PRICE'][df_p['TYPE_CODE'] == 'C5'].sum()
print('The toal expenses for domestic flights for previous month equals SAR {:,}'.format(domExpP))

In [ ]:
domExp = df['PRICE'][df['TYPE_CODE'] == 'C5'].sum()
print('The toal expenses for domestic flights for current month equals SAR {:,}'.format(domExp))

##### Handling Charges:

In [ ]:
handlingChargesP = df_p['PRICE'][(df_p['TYPE_CODE'] == 'C5')&(df_p.ITEM_CODE.isin(['#HAH','#HAGG','#HAFF','#HAAM', '#HAAL', '#HAA', '#HAB', '#HAE', '#HAD', '#HAF', '#HAG', '#HAI']))].sum()
handlingPerPaxP = handlingChargesP/paxCountP
print('The total handling charges for previous month equals to {:,} and handling per pax equals to {}'.format(handlingChargesP, np.round(handlingPerPaxP, 2)))

In [ ]:
handlingCharges = df['PRICE'][(df['TYPE_CODE'] == 'C5')&(df.ITEM_CODE.isin(['#HAAM', '#HAAL', '#HAA', '#HAB', '#HAE', '#HAD', '#HAF', '#HAG', '#HAI']))].sum()
handlingPerPax = handlingCharges/paxCount
print('The total handling charges for current month equals to {:,} and handling per pax equals to {}'.format(handlingCharges, np.round(handlingPerPax, 2)))

##### Expenses per Class for Previous Month:

In [ ]:
fc_exp_p = df_p['PRICE'][(df_p['TYPE_CODE'] == 'C5') & (df_p['CLASS_CODE'] == 'FC')].sum()
print('First class expenses equals to SAR {:,}'.format(fc_exp_p))

bc_exp_p = df_p['PRICE'][(df_p['TYPE_CODE'] == 'C5') & (df_p['CLASS_CODE'] == 'JC')].sum()
print('Bsiness class expenses equals to SAR {:,}'.format(bc_exp_p))

yc_exp_p = df_p['PRICE'][(df_p['TYPE_CODE'] == 'C5') & (df_p['CLASS_CODE'] == 'YC')].sum() - handlingChargesP
print('Economy class expenses equals to SAR {:,}'.format(yc_exp_p))

##### Expenses per Class for Current month:

In [ ]:
fc_exp = df['PRICE'][(df['TYPE_CODE'] == 'C5') & (df['CLASS_CODE'] == 'FC')].sum()
print('First class expenses equals to SAR {:,}'.format(fc_exp))

bc_exp = df['PRICE'][(df['TYPE_CODE'] == 'C5') & (df['CLASS_CODE'] == 'JC')].sum()
print('Bsiness class expenses equals to SAR {:,}'.format(bc_exp))

yc_exp = df['PRICE'][(df['TYPE_CODE'] == 'C5') & (df['CLASS_CODE'] == 'YC')].sum() - handlingCharges
print('Economy class expenses equals to SAR {:,}'.format(yc_exp))

##### Cost per Passenger (CPP) for Previous Month:

In [ ]:
fc_cpp_p = fc_exp_p / fc_count_p
fc_cpp_h_p = fc_cpp_p + handlingPerPaxP
print('First class meal cpp equals to SAR {} and SAR {} including handling charges.'.format(np.round(fc_cpp_p, 2), np.round(fc_cpp_h_p, 2)))

bc_cpp_p = bc_exp_p / bc_count_p
bc_cpp_h_p = bc_cpp_p + handlingPerPaxP
print('Business class meal cpp equals to SAR {} and SAR {} including handling charges.'.format(np.round(bc_cpp_p, 2), np.round(bc_cpp_h_p, 2)))

yc_cpp_p = yc_exp_p / yc_count_p
yc_cpp_h_p = yc_cpp_p + handlingPerPaxP
print('Economy class meal cpp equals to SAR {} and SAR {} including handling charges.'.format(np.round(yc_cpp_p, 2), np.round(yc_cpp_h_p, 2)))

avgCppP = domExpP/paxCountP
avgCppNoHacP = avgCppP - handlingPerPaxP
print('The average CPP equals SAR {} and average meal CPP equals SAR {} .'.format(np.round(avgCppP,2), np.round(avgCppNoHacP, 2)))

##### Cost per Passenger (CPP) for Current Month:

In [ ]:
fc_cpp = fc_exp / fc_count
fc_cpp_h = fc_cpp + handlingPerPax
print('First class meal cpp equals to SAR {} and SAR {} including handling charges.'.format(np.round(fc_cpp, 2), np.round(fc_cpp_h, 2)))

bc_cpp = bc_exp / bc_count
bc_cpp_h = bc_cpp + handlingPerPax
print('Business class meal cpp equals to SAR {} and SAR {} including handling charges.'.format(np.round(bc_cpp, 2), np.round(bc_cpp_h, 2)))

yc_cpp = yc_exp / yc_count
yc_cpp_h = yc_cpp + handlingPerPax
print('Economy class meal cpp equals to SAR {} and SAR {} including handling charges.'.format(np.round(yc_cpp, 2), np.round(yc_cpp_h, 2)))

avgCpp = domExp/paxCount
avgCppNoHac = avgCpp - handlingPerPax
print('The average CPP equals SAR {} and average meal CPP equals SAR {} .'.format(np.round(avgCpp,2), np.round(avgCppNoHac, 2)))

### What are the monthly changes and their impact?

### Create Menu Frames by Service:

In [ ]:
# make a copy of the invoice df and df_p
df_three = df.copy()
df_p_three = df_p.copy()

##### Beverage Flights:

In [ ]:
# Define variables for filteration  
handling_item_codes =['#HAH','#HAGG','#HAFF','#HAAM', '#HAAL', '#HAA', '#HAB', '#HAE', '#HAD', '#HAF', '#HAG', '#HAI']
# Flights that supposed to be beverage but follows meal policy
meal_exception = ['RUH-MED-RUH', 'RUH-YNB-RUH', 'RUH-ABT-RUH', 'RUH-BHH-RUH', 'RUH-TIF-RUH' ]
# Flights that supposed to be meal but follows beverage policy
beverage_exception = ['JED-ELQ-JED', 'JED-HAS-JED', 'JED-EAM-JED', 'JED-WAE-JED', 'JED-GIZ-JED', 'RUH-RAH-RUH' ]
# Add route column for comparsion purposes 
df_three['ROUTE'] = df_three['FROM_CITY'] + '-' + df_three['TO_CITY'] + '-' + df_three['FROM_CITY'] 
df_p_three['ROUTE'] = df_three['FROM_CITY'] + '-' + df_three['TO_CITY'] + '-' + df_three['FROM_CITY'] 

#### YC beverage flight impact:

In [ ]:
# create beverages data frame for YC
bev_frame_yc = df_three[(df_three['Block Time'] <= 89) & (df_three['CLASS_CODE'] == 'YC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_three.ROUTE.isin(meal_exception)) | (df_three.ROUTE.isin(beverage_exception))]
bev_frame_p_yc = df_p_three[(df_p_three['Block Time'] <= 89) & (df_p_three['CLASS_CODE'] == 'YC')& ~(df_p_three.ITEM_CODE.isin(handling_item_codes))  & ~(df_p_three.ROUTE.isin(meal_exception)) | (df_p_three.ROUTE.isin(beverage_exception))]

# Remove duplicates rows
bev_frame_yc.drop_duplicates(subset='ITEM_CODE', inplace= True)
bev_frame_p_yc.drop_duplicates(subset='ITEM_CODE', inplace= True)

# drop unneeded columns
bev_frame_yc = bev_frame_yc.drop(columns=['CDN_NO', 'DBL_DATE','FLT_NO','ETD', 'FROM_CITY', 'TO_CITY', 'CLASS_CODE', 'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'Block Time', 'ROUTE'])
bev_frame_p_yc = bev_frame_p_yc.drop(columns=['CDN_NO', 'DBL_DATE','FLT_NO','ETD', 'FROM_CITY', 'TO_CITY', 'CLASS_CODE', 'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'Block Time', 'ROUTE'])

# reset the frames indexs
bev_frame_p_yc.reset_index(drop=True, inplace=True)
bev_frame_yc.reset_index(drop=True, inplace=True)

# print previous month dataframe
print('Previous month menu:')
display(bev_frame_p_yc)
# print current month dataframe
print('Current month menu:')
display(bev_frame_yc)

# compare this month's items by last month's items, using the isin() function
changes_list = bev_frame_yc[~(bev_frame_yc.isin(bev_frame_p_yc))]
# remove empty rows using the dropna() function
changes_list.dropna(subset=['ITEM_CODE', 'SPECS_NO'], inplace=True)

#Create another frame for the new month to calculate the impact
bev_frame_yc_im = df_three[(df_three['Block Time'] <= 89) & (df_three['CLASS_CODE'] == 'YC') & ~(df_three.ITEM_CODE.isin(handling_item_codes))  & ~(df_three.ROUTE.isin(meal_exception)) | (df_three.ROUTE.isin(beverage_exception))]



#Get the total cost of each new item
changes_list['TOTAL_COST'] = 0 # Add impact column to change_list
changes_list['CPP_IMPACT'] = 0 # Add impact column to change_list
bev_yc_cost_im = 0  # place holder for total cost impact for each item
for x,z in enumerate(changes_list['SPECS_NO']): # loop to get the specs number for new items
    for i, y in enumerate(bev_frame_yc_im['PRICE'][bev_frame_yc_im['SPECS_NO'] == z]): #get the impact on each flight and add it to bev_yc_cost_im
        bev_yc_cost_im = bev_yc_cost_im + y
    # claculate the CPP for the current spec number    
    cpp_im = bev_yc_cost_im / paxCount
    # Add the total impact for the current spec number
    changes_list['TOTAL_COST'].iloc[x] = bev_yc_cost_im
    # Add the CPP impact to the current spec number
    changes_list['CPP_IMPACT'].iloc[x] = cpp_im
    # rest place holders to zero
    cpp_im = 0
    bev_yc_cost_im = 0

if changes_list['TOTAL_COST'].sum() > 0:
    print('The following is the impact breakdown:')
    display(changes_list)
    print('Total cost impact on beverage flights economy equals SAR {:,} and its impact on CPP equals SAR {:,}'.format(changes_list['TOTAL_COST'].sum(), changes_list['CPP_IMPACT'].sum()))
else:
    print('No changes observed')

##### BC beverage flight impact:

In [ ]:
# create beverages data frame for BC
bev_frame_bc = df_three[(df_three['Block Time'] <= 89) & (df_three['CLASS_CODE'] == 'JC') & (df_three.ROUTE.isin(beverage_exception)) & ~(df_three.ROUTE.isin(meal_exception))]
bev_frame_p_bc = df_p_three[(df_p_three['Block Time'] <= 89) & (df_p_three['CLASS_CODE'] == 'JC') & (df_three.ROUTE.isin(beverage_exception)) & ~(df_three.ROUTE.isin(meal_exception))]

# Remove duplicates rows
bev_frame_bc.drop_duplicates(subset='ITEM_CODE', inplace= True)
bev_frame_p_bc.drop_duplicates(subset='ITEM_CODE', inplace= True)

# drop unneeded columns
bev_frame_bc = bev_frame_bc.drop(columns=['CDN_NO', 'DBL_DATE','FLT_NO','ETD', 'FROM_CITY', 'TO_CITY', 'CLASS_CODE', 'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'Block Time', 'ROUTE'])
bev_frame_p_bc = bev_frame_p_bc.drop(columns=['CDN_NO', 'DBL_DATE','FLT_NO','ETD', 'FROM_CITY', 'TO_CITY', 'CLASS_CODE', 'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'Block Time', 'ROUTE'])

# reset the frames indexs
bev_frame_p_bc.reset_index(drop=True, inplace=True)
bev_frame_bc.reset_index(drop=True, inplace=True)


# print previous month dataframe
print('Previous month menu:')
display(bev_frame_p_bc)

# print current month dataframe
print('Current month menu:')
display(bev_frame_bc)

# compare this month's items by last month's items, using the isin() function
changes_list_bb = bev_frame_bc[~(bev_frame_bc.isin(bev_frame_p_bc))]
# remove empty rows using the dropna() function
changes_list_bb.dropna(subset=['ITEM_CODE', 'SPECS_NO'], inplace=True)

#Create another frame for the new month to calculate the impact
bev_frame_bc_im = df_three[(df_three['Block Time'] <= 89) & (df_three['CLASS_CODE'] == 'JC') & (df_three.ROUTE.isin(beverage_exception)) & ~(df_three.ROUTE.isin(meal_exception))]

#Get the total cost of each new item
changes_list_bb['TOTAL_COST'] = 0 # Add impact column to change_list
changes_list_bb['CPP_IMPACT'] = 0 # Add impact column to change_list
bev_bc_cost_im = 0  # place holder for total cost impact for each item
for x,z in enumerate(changes_list_bb['SPECS_NO']): # loop to get the specs number for new items
    for i, y in enumerate(bev_frame_bc_im['PRICE'][bev_frame_bc_im['SPECS_NO'] == z]): #get the impact on each flight and add it to bev_yc_cost_im
        bev_bc_cost_im = bev_bc_cost_im + y
    # claculate the CPP for the current spec number    
    cpp_im = bev_bc_cost_im / paxCount
    # Add the total impact for the current spec number
    changes_list_bb['TOTAL_COST'].iloc[x] = bev_bc_cost_im
    # Add the CPP impact to the current spec number
    changes_list_bb['CPP_IMPACT'].iloc[x] = cpp_im
    # rest place holders to zero
    cpp_im = 0
    bev_bc_cost_im = 0


if changes_list_bb['TOTAL_COST'].sum() > 0:
    print('The following is the impact breakdown:')
    display(changes_list_bb)
    print('Total cost impact on beverage flights business equals SAR {:,} and its impact on CPP equals SAR {:,}'.format(changes_list_bb['TOTAL_COST'].sum(), changes_list_bb['CPP_IMPACT'].sum()))
else:
    print('No changes observed')

#### YC meal flights impact:

In [ ]:
# create meals data frame for YC
meal_frame_yc = df_three[((df_three['Block Time'] > 89) & (df_three['CLASS_CODE'] == 'YC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_three.ROUTE.isin(beverage_exception))) | ((df_three.ROUTE.isin(meal_exception)) & (df_three['CLASS_CODE'] == 'YC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)))]
meal_frame_p_yc = df_p_three[((df_p_three['Block Time'] > 89) & (df_p_three['CLASS_CODE'] == 'YC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_p_three.ROUTE.isin(beverage_exception))) | ((df_p_three.ROUTE.isin(meal_exception)) & (df_p_three['CLASS_CODE'] == 'YC') & ~(df_p_three.ITEM_CODE.isin(handling_item_codes)))]

# Remove duplicates rows
meal_frame_yc.drop_duplicates(subset='ITEM_CODE', inplace= True)
meal_frame_p_yc.drop_duplicates(subset='ITEM_CODE', inplace= True)

# drop unneeded columns
meal_frame_yc = meal_frame_yc.drop(columns=['CDN_NO', 'DBL_DATE','FLT_NO','ETD', 'FROM_CITY', 'TO_CITY', 'CLASS_CODE', 'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'Block Time', 'ROUTE'])
meal_frame_p_yc = meal_frame_p_yc.drop(columns=['CDN_NO', 'DBL_DATE','FLT_NO','ETD', 'FROM_CITY', 'TO_CITY', 'CLASS_CODE', 'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'Block Time', 'ROUTE'])

# reset the frames indexs
meal_frame_p_yc.reset_index(drop=True, inplace=True)
meal_frame_yc.reset_index(drop=True, inplace=True)

# sort menu
#meal_frame_p_yc.sort_values(by='ITEM_DESCRIPTION', inplace=True)
#meal_frame_yc.sort_values(by='ITEM_DESCRIPTION', inplace=True)

# print previous month dataframe
print('Previous month menu:')
display(meal_frame_p_yc)

# print current month dataframe
print('Current month menu:')
display(meal_frame_yc)

# compare this month's items by last month's items, using the isin() function
changes_list_ym = meal_frame_yc[~(meal_frame_yc['SPECS_NO'].isin(meal_frame_p_yc['SPECS_NO']))]
# remove empty rows using the dropna() function
changes_list_ym.dropna(subset=['ITEM_CODE', 'SPECS_NO'], inplace=True)

#Create another frame for the new month to calculate the impact
meal_frame_yc_im = df_three[((df_three['Block Time'] > 89) & (df_three['CLASS_CODE'] == 'YC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_three.ROUTE.isin(beverage_exception))) | ((df_three.ROUTE.isin(meal_exception)) & (df_three['CLASS_CODE'] == 'YC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)))]

#Get the total cost of each new item
changes_list_ym['TOTAL_COST'] = 0 # Add impact column to change_list
changes_list_ym['CPP_IMPACT'] = 0 # Add impact column to change_list
meal_yc_cost_im = 0  # place holder for total cost impact for each item
for x,z in enumerate(changes_list_ym['SPECS_NO']): # loop to get the specs number for new items
    for i, y in enumerate(meal_frame_yc_im['PRICE'][meal_frame_yc_im['SPECS_NO'] == z]): #get the impact on each flight and add it to bev_yc_cost_im
        meal_yc_cost_im = meal_yc_cost_im + y
    # claculate the CPP for the current spec number    
    cpp_im = meal_yc_cost_im / paxCount
    # Add the total impact for the current spec number
    changes_list_ym['TOTAL_COST'].iloc[x] = meal_yc_cost_im
    # Add the CPP impact to the current spec number
    changes_list_ym['CPP_IMPACT'].iloc[x] = cpp_im
    # rest place holders to zero
    cpp_im = 0
    bev_bc_cost_im = 0


if changes_list_ym['TOTAL_COST'].sum() > 0:
    print('The following is the impact breakdown:')
    display(changes_list_ym)
    print('Total cost impact on meal flights economy equals SAR {:,} and its impact on CPP equals SAR {:,}'.format(changes_list_ym['TOTAL_COST'].sum(), changes_list_ym['CPP_IMPACT'].sum()))
else:
    print('No changes observed')

#### BC meal flights impact:

In [ ]:
# create meals data frame for BC
meal_frame_bc = df_three[(df_three['Block Time'] > 89) & (df_three['CLASS_CODE'] == 'JC') & ~(df_three.ROUTE.isin(beverage_exception)) & (df_three.ROUTE.isin(meal_exception))]
meal_frame_p_bc = df_p_three[(df_p_three['Block Time'] > 89) & (df_p_three['CLASS_CODE'] == 'JC') & ~(df_p_three.ROUTE.isin(beverage_exception)) & (df_p_three.ROUTE.isin(meal_exception))]

# Remove duplicates rows
meal_frame_bc.drop_duplicates(subset='ITEM_CODE', inplace= True)
meal_frame_p_bc.drop_duplicates(subset='ITEM_CODE', inplace= True)

# drop unneeded columns
meal_frame_bc = meal_frame_bc.drop(columns=['CDN_NO', 'DBL_DATE','FLT_NO','ETD', 'FROM_CITY', 'TO_CITY', 'CLASS_CODE', 'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'Block Time', 'ROUTE'])
meal_frame_p_bc = meal_frame_p_bc.drop(columns=['CDN_NO', 'DBL_DATE','FLT_NO','ETD', 'FROM_CITY', 'TO_CITY', 'CLASS_CODE', 'CAPACITY', 'NO_PAX', 'PRICE', 'TYPE_CODE', 'Block Time', 'ROUTE'])

# reset the frames indexs
meal_frame_p_bc.reset_index(drop=True, inplace=True)
meal_frame_bc.reset_index(drop=True, inplace=True)


# print previous month dataframe
print('Previous month menu:')
display(meal_frame_p_bc)

# print current month dataframe
print('Current month menu:')
display(meal_frame_bc)

# compare this month's items by last month's items, using the isin() function
changes_list_bm = meal_frame_bc[~(meal_frame_bc.isin(meal_frame_p_bc))]
# remove empty rows using the dropna() function
changes_list_bm.dropna(subset=['ITEM_CODE', 'SPECS_NO'], inplace=True)

#Create another frame for the new month to calculate the impact
meal_frame_bc_im = df_three[(df_three['Block Time'] > 89) & (df_three['CLASS_CODE'] == 'JC') & ~(df_three.ROUTE.isin(beverage_exception)) & (df_three.ROUTE.isin(meal_exception))]

#Get the total cost of each new item
changes_list_bm['TOTAL_COST'] = 0 # Add impact column to change_list
changes_list_bm['CPP_IMPACT'] = 0 # Add impact column to change_list
meal_bc_cost_im = 0  # place holder for total cost impact for each item
for x,z in enumerate(changes_list_bm['SPECS_NO']): # loop to get the specs number for new items
    for i, y in enumerate(meal_frame_bc_im['PRICE'][meal_frame_bc_im['SPECS_NO'] == z]): #get the impact on each flight and add it to bev_yc_cost_im
        meal_bc_cost_im = meal_bc_cost_im + y
    # claculate the CPP for the current spec number    
    cpp_im = meal_bc_cost_im / paxCount
    # Add the total impact for the current spec number
    changes_list_bm['TOTAL_COST'].iloc[x] = meal_bc_cost_im
    # Add the CPP impact to the current spec number
    changes_list_bm['CPP_IMPACT'].iloc[x] = cpp_im
    # rest place holders to zero
    cpp_im = 0
    meal_bc_cost_im = 0


if changes_list_bm['TOTAL_COST'].sum() > 0:
    print('The following is the impact breakdown:')
    display(changes_list_bm)
    print('Total cost impact on meal flights Business class equals SAR {:,} and its impact on CPP equals SAR {:,}'.format(changes_list_bm['TOTAL_COST'].sum(), changes_list_bm['CPP_IMPACT'].sum()))
else:
    print('No changes observed')

In [ ]:
summerSchedule[(summerSchedule['FROM_CITY'] == 'JED') & (summerSchedule['TO_CITY'] == 'RUH')]

### TOP5 YC menu

In [ ]:
df[(df['FROM_CITY'] == 'JED') & (df['TO_CITY'] == 'RUH') & (df['CLASS_CODE'] == 'YC') & (df['FLT_NO'] == '1052') & (df['DBL_DATE'] == '2021-03-12')]

In [ ]:
x = df_three[((df_three['Block Time'] > 89) & (df_three['CLASS_CODE'] == 'YC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)) & ~(df_three.ROUTE.isin(beverage_exception))) | ((df_three.ROUTE.isin(meal_exception)) & (df_three['CLASS_CODE'] == 'YC') & ~(df_three.ITEM_CODE.isin(handling_item_codes)))]

In [ ]:
x['ROUTE'].unique()

In [ ]:
x['CLASS_CODE'].unique()

In [ ]:
y = df_three[(df_three['Block Time'] <= 89) & (df_three['CLASS_CODE'] == 'YC')]
y = y[~(y.ITEM_CODE.isin(handling_item_codes))]
y = y[(y.ROUTE.isin(beverage_exception)) | ~(y.ROUTE.isin(meal_exception))]

In [ ]:
y['CLASS_CODE'].unique()

In [ ]:
meal_exception = ['RUH-MED-RUH', 'RUH-YNB-RUH', 'RUH-ABT-RUH', 'RUH-BHH-RUH', 'RUH-TIF-RUH' ]
# Flights that supposed to be meal but follows beverage policy
beverage_exception = ['JED-ELQ-JED', 'JED-HAS-JED', 'JED-EAM-JED', 'JED-WAE-JED', 'JED-GIZ-JED', 'RUH-RAH-RUH' ]